# Text to Text  Explanation: Abstractive Summarization Example

This notebook demonstrates use of generating model explanations for a text to text scenario on a pretrained transformer model. Below we demonstrate the process of generating explanations for a pretrained model distilbart on the Extreme Summarization (XSum) Dataset provided by hugging face (https://huggingface.co/sshleifer/distilbart-xsum-12-6). 

The first example only needs the model and tokenizer and we use the model decoder to generate log odds of the output tokens to be explained. In the second example, we demonstrate the use of how to generate expplanations for model in the form of an api/fucntion (input->text and output->text). In this case we need to approximate the log odds by using a text similarity model. The underlying explainer used to compute the shap values is the partition explainer.

In [17]:
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, T5Tokenizer
import torch
# from datasets import load_dataset
import shap

### Load model and tokenizer

In [18]:
tokenizer = T5Tokenizer.from_pretrained("t5-base")
model =  AutoModelForSeq2SeqLM.from_pretrained("t5-base").cuda()

### Load data

In [19]:
tokenizer

PreTrainedTokenizer(name_or_path='t5-base', vocab_size=32100, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_id_44>', '<extra_id_45>',

In [14]:
from IPython.core.debugger import set_trace 

In [27]:
tokenizer('I hate you!!!')

{'input_ids': [27, 5591, 25, 3158, 1], 'attention_mask': [1, 1, 1, 1, 1]}

In [3]:
dataset = load_dataset('xsum',split='train')

Using custom data configuration default
Reusing dataset xsum (C:\Users\ryserrao\.cache\huggingface\datasets\xsum\default\1.1.0\128741c17b7a4c939dbf844a75a5e83deadd07deaf4b2eda2056ed8eebdb03ae)


In [4]:
# slice inputs from dataset to run model inference on
s = dataset['document'][0:1]

### Create an explainer object

In [20]:
# set_trace()
explainer = shap.Explainer(model,tokenizer)

### Compute shap values

In [24]:
s = ['Summarization: I enjoy walking with my cute dog']

In [25]:
shap_values = explainer(s)

> /ext3/miniconda3/lib/python3.8/site-packages/shap/explainers/_explainer.py(255)__call__()
    253             feature_names = [[] for _ in range(len(args))]
    254         for row_args in show_progress(zip(*args), num_rows, self.__class__.__name__+" explainer", silent):
2-> 255             row_result = self.explain_row(
    256                 *row_args, max_evals=max_evals, main_effects=main_effects, error_bounds=error_bounds,
    257                 batch_size=batch_size, outputs=outputs, silent=silent, **kwargs

ipdb> c
> /ext3/miniconda3/lib/python3.8/site-packages/shap/explainers/_explainer.py(255)__call__()
    253             feature_names = [[] for _ in range(len(args))]
    254         for row_args in show_progress(zip(*args), num_rows, self.__class__.__name__+" explainer", silent):
2-> 255             row_result = self.explain_row(
    256                 *row_args, max_evals=max_evals, main_effects=main_effects, error_bounds=error_bounds,
    257                 batch_siz

### Visualize shap explanations

In [26]:
shap.plots.text(shap_values)

### API

Below we demonstrate generating explanations for a model which is an api/function. Since this is a model agnostic case, we use a text similarity model to approximate log odds of generating output text which is used for computing shap explanations.

In [8]:
# Define function
def f(x):
    inputs = tokenizer(x.tolist(), return_tensors="pt", padding=True).to('cuda')
    with torch.no_grad():
        out = model.generate(**inputs)
    sentence = [tokenizer.decode(g, skip_special_tokens=True) for g in out]
    return np.array(sentence)

For a model agnostic case, we wrap the model to be explained with the shal.models.TeacherForcing class and define the text similarity model and tokenizer. The TeacherForcing class uses the similarity model to approximate the log odds of generating the output text from the model(function->f)

We also have to define a Text masker and define mask_token="..." and pass collapse_mask_token=True, which then cues the algorithm to use text infilling while masking

In [9]:
# wrap model with TeacherForcingLogits class
teacher_forcing_model = shap.models.TeacherForcing(f, similarity_model=model, similarity_tokenizer=tokenizer)
# create a Text masker
masker = shap.maskers.Text(tokenizer, mask_token = "...", collapse_mask_token=True)

### Create an explainer object using wrapped model and Text masker

In [10]:
explainer_model_agnostic = shap.Explainer(teacher_forcing_model, masker)

explainers.Partition is still in an alpha state, so use with caution...


### Compute shap values

In [11]:
shap_values_model_agnostic = explainer_model_agnostic(s)

Partition explainer: 2it [00:31, 15.99s/it]                                                                            


### Visualize shap explanations

In [12]:
shap.plots.text(shap_values_model_agnostic)